In [6]:
import numpy as np
import pandas as pd
from netCDF4 import Dataset, num2date

# --- Open file ---
ds = Dataset("./data/2902273_Sprof.nc")

# --- Core variables ---
lat = ds.variables['LATITUDE'][:]      # (Nprofiles,)
lon = ds.variables['LONGITUDE'][:]
time_var = ds.variables['JULD'][:]
time_units = ds.variables['JULD'].units
time = num2date(time_var, time_units)  # datetime objects

temp = ds.variables['TEMP'][:]         # (Nprofiles, Nlevels)
sal  = ds.variables['PSAL'][:]
pres = ds.variables['PRES'][:]

# --- Dimensions ---
Nprofiles, Nlevels = temp.shape

lat_expanded  = np.repeat(lat, Nlevels)
lon_expanded  = np.repeat(lon, Nlevels)
time_expanded = np.repeat(time, Nlevels)

# Flatten core vars
temp_flat = temp.reshape(-1)
sal_flat  = sal.reshape(-1)
pres_flat = pres.reshape(-1)

# --- Collect data ---
data = {
    "time": time_expanded,
    "lat": lat_expanded,
    "lon": lon_expanded,
    "pressure": pres_flat,
    "temperature": temp_flat,
    "salinity": sal_flat
}

# --- Try to add BGC variables if present ---
bgc_vars = ["DOXY", "CHLA", "BBP700", "NITRATE", "PH_IN_SITU_TOTAL"]

for var in bgc_vars:
    if var in ds.variables:
        arr = ds.variables[var][:]
        if arr.ndim == 2:  # (Nprofiles, Nlevels)
            data[var.lower()] = arr.reshape(-1)  # flatten
        else:
            print(f"Skipping {var}, unexpected shape: {arr.shape}")

# --- Build DataFrame ---
df = pd.DataFrame(data)

# --- Save & preview ---
df.to_csv("bcg_data_latest1.csv", index=False)
print("Saved with columns:", df.columns.tolist())
print(df.head())


Saved with columns: ['time', 'lat', 'lon', 'pressure', 'temperature', 'salinity', 'doxy', 'chla', 'bbp700']
                         time        lat        lon  pressure  temperature  \
0  2019-05-06 01:51:47.999986  17.567539  68.389618      4.32    29.533701   
1  2019-05-06 01:51:47.999986  17.567539  68.389618      5.96    29.346901   
2  2019-05-06 01:51:47.999986  17.567539  68.389618      8.00    29.224701   
3  2019-05-06 01:51:47.999986  17.567539  68.389618      9.31    29.199083   
4  2019-05-06 01:51:47.999986  17.567539  68.389618     10.02    29.185200   

    salinity        doxy   chla   bbp700  
0  36.653801         NaN    NaN      NaN  
1  36.628201         NaN    NaN      NaN  
2  36.619099         NaN    NaN      NaN  
3  36.616116  230.829086  0.072  0.00085  
4  36.614498         NaN    NaN      NaN  
